In [ ]:
import numpy as np
import cv2
from tqdm.auto import tqdm

In [ ]:
def read_video(video_path: str) -> np.array:
    cap = cv2.VideoCapture(video_path)
    frame_count = int(cap.get(cv2.CAP_PROP_FRAME_COUNT))

    frames = []
    for _ in tqdm(range(frame_count), desc='Reading video'):
        ret, frame = cap.read()
        if not ret:
            break

        # converting to grayscale for simplicity
        frames.append(cv2.cvtColor(frame, cv2.COLOR_BGR2GRAY))

    cap.release()
    frames = np.array(frames)

    return frames

In [ ]:
def build_laplacian_pyramid(frame, levels=3):
    pyramid = []
    current_frame = frame
    for _ in range(levels - 1):
        down = cv2.pyrDown(current_frame)
        up = cv2.pyrUp(down, dstsize=(current_frame.shape[1], current_frame.shape[0]))
        laplacian = cv2.subtract(current_frame, up)
        pyramid.append(laplacian)
        current_frame = down
    pyramid.append(current_frame)
    return pyramid

In [ ]:
def temporal_filter(frames: np.array, band_pass_low, band_pass_high, fps):
    fft = np.fft.fft(frames, axis=0)
    frequencies = np.fft.fftfreq(frames.shape[0], d=1 / fps)
    mask = np.logical_and(frequencies >= band_pass_low, frequencies <= band_pass_high)
    fft[~mask] = 0
    filtered = np.real(np.fft.ifft(fft, axis=0))
    return filtered

In [ ]:
def amplify_motion(pyramid, alpha):
    for level in range(len(pyramid)):
        # pyramid[level] *= alpha
        pyramid[level] = cv2.convertScaleAbs(pyramid[level] * alpha, alpha=1 / 255.0)
    return pyramid

In [ ]:
def reconstruct_from_laplacian(pyramid):
    frame = pyramid[-1]
    levels = len(pyramid)
    for level in range(levels - 1, 0, -1):
        frame = cv2.pyrUp(frame, dstsize=(pyramid[level - 1].shape[1], pyramid[level - 1].shape[0]))
        frame = cv2.add(frame, pyramid[level - 1])
    return frame

In [ ]:
def eulerian_magnification(frames: np.array, alpha, freq_low, freq_high, fps):
    magnified_frames = []

    # Display progress while processing with tqdm
    for frame in tqdm(frames, desc='Processing frames'):
        pyramid = build_laplacian_pyramid(frame)
        filtered = [temporal_filter(layer, freq_low, freq_high, fps) for layer in pyramid]
        amplified = amplify_motion(filtered, alpha)
        reconstructed_frame = reconstruct_from_laplacian(amplified)
        magnified_frames.append(reconstructed_frame)
    return magnified_frames

In [ ]:
video_path = '/Volumes/Patrick/Proband06/Logitech HD Pro Webcam C920.avi'
frames = read_video(video_path)
magnified_frames = eulerian_magnification(frames, 10, 0.4, 3, 30)

In [ ]:
# Display one of the magnified frames with mathplotlib
import matplotlib.pyplot as plt

plt.imshow(magnified_frames[0], cmap='gray')
plt.show()

In [ ]:
# Save the magnified video
output_video_path = '../magnified.avi'
fourcc = cv2.VideoWriter_fourcc(*'XVID')
out = cv2.VideoWriter(output_video_path, fourcc, 30, (640, 480))

for inx in tqdm(range(len(magnified_frames)), desc='Writing video'):
    frame = frames[inx]
    magnified_frame = magnified_frames[inx]
    
    # Magnify the frame by the magnified frame
    magnified_frame = cv2.cvtColor(magnified_frame, cv2.COLOR_GRAY2BGR)
    
    # Add the magnified frame to the original frame
    # frame = cv2.cvtColor(frame, cv2.COLOR_GRAY2BGR)
    frame = cv2.addWeighted(frame, 0.5, magnified_frame, 0.5, 0)
    
    out.write(cv2.cvtColor(frame, cv2.COLOR_GRAY2BGR))
out.release()